In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import sys
sys.path.append('/home/sedroc/Bachelor/FEPCEU')
from enhanced_experiment import EnhancedTimeSeriesExperiment
from models.arima import Arima
from models.baseline import Baseline
from models.timegpt import TimeGPT
from models.lgbm import LGBM
from models.neuralproph import Neuralprophet

from data.data_retriever import Dataretreiver

In [17]:
data = Dataretreiver(debug=True, reduce='pca_pure')
df = data.combined

Filling missing values
Start date: 2024-01-01 00:00:00
End date: 2024-12-31 23:00:00
Total expected hours: 8784
Hours in original data: 8783
Missing hours filled: 1
Hours in complete data: 8784
Expected hours for the year: 8784
Top columns selected: ['sun_area_36', 'sun_area_21', 'sun_area_31', 'sun_area_6', 'sun_area_5', 'sun_area_1', 'sun_area_16', 'sun_area_11', 'sun_area_4', 'sun_area_22']
Total variance explained with the 10 columns: 98.06%
Top columns selected: ['wind_area_5', 'wind_area_36', 'wind_area_31', 'wind_area_15', 'wind_area_29', 'wind_area_40', 'wind_area_1', 'wind_area_20', 'wind_area_2', 'wind_area_25']
Total variance explained with the 10 columns: 96.63%
Top columns selected: ['temp_area_25', 'temp_area_15', 'temp_area_36', 'temp_area_40', 'temp_area_14', 'temp_area_11', 'temp_area_39', 'temp_area_20', 'temp_area_19', 'temp_area_37']
Total variance explained with the 10 columns: 98.95%


In [ ]:
# Define models
models = {
    #'BASELINE': Baseline(),
    'LGBM': LGBM(),
    #'ARIMA': Arima(order=[1,0,1], seasonal_order=[1,1,1,24]),
    #'TimeGPT': TimeGPT(),
    #'NEURALPROPHET': Neuralprophet()
}

In [31]:
# Initialize the experiment
experiment = EnhancedTimeSeriesExperiment(
    models=models,
    target_column='price',
    forecast_horizon=192,
    n_splits=1,
    step_size=192
)

# Run the experiments with these custom combinations
experiment.run_feature_group_experiments(
    df=df,
    add_all_columns=True,
    add_base_columns=False,
    manipulate=True,
    manipulate_factor_sun=0.5,
    manipulate_factor_wind=0.5
)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: h
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Querying model metadata...


Identified area columns: {'sun': {36: 'sun_area_36', 21: 'sun_area_21', 31: 'sun_area_31', 6: 'sun_area_6', 5: 'sun_area_5', 1: 'sun_area_1', 16: 'sun_area_16', 11: 'sun_area_11', 4: 'sun_area_4', 22: 'sun_area_22'}, 'wind': {5: 'wind_area_5', 36: 'wind_area_36', 31: 'wind_area_31', 15: 'wind_area_15', 29: 'wind_area_29', 40: 'wind_area_40', 1: 'wind_area_1', 20: 'wind_area_20', 2: 'wind_area_2', 25: 'wind_area_25'}}
Cross-validation splits with sliding window:
Split 1: Train start = 2024-01-01 00:00:00 (index 0), Train end = 2024-06-29 20:00:00 (index 4340), Test end = 2024-07-07 19:00:00 (index 4531), Training size = 4340

Running experiment for feature group: all_areas
Using columns: ['sun_area_36', 'sun_area_21', 'sun_area_31', 'sun_area_6', 'sun_area_5', 'sun_area_1', 'sun_area_16', 'sun_area_11', 'sun_area_4', 'sun_area_22', 'wind_area_5', 'wind_area_36', 'wind_area_31', 'wind_area_15', 'wind_area_29', 'wind_area_40', 'wind_area_1', 'wind_area_20', 'wind_area_2', 'wind_area_25']


INFO:nixtla.nixtla_client:Using historical exogenous features: ['sun_area_36', 'sun_area_21', 'sun_area_31', 'sun_area_6', 'sun_area_5', 'sun_area_1', 'sun_area_16', 'sun_area_11', 'sun_area_4', 'sun_area_22', 'wind_area_5', 'wind_area_36', 'wind_area_31', 'wind_area_15', 'wind_area_29', 'wind_area_40', 'wind_area_1', 'wind_area_20', 'wind_area_2', 'wind_area_25', 'ds', 'unique_id']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


In [30]:
# Get a table of results
results_df = experiment.summarize_feature_group_results(metric='MAE')
print(results_df)
results_df = experiment.summarize_feature_group_results(metric='RMSE')
print(results_df)

# Plot the results
experiment.plot_feature_group_results(metric='MAE')

                     TimeGPT
all_areas  599.6606 ± 0.0000
                     TimeGPT
all_areas  646.7547 ± 0.0000


In [ ]:
# Plot predictions for the 'sun_areas_1_2_3' feature group
experiment.plot_feature_group_predictions(
    df=df,
    feature_group='all_areas',
    fold_index=0  # First CV fold
)